## Windowed aggregations using spark streaming and ingestion to the online feature store

### **Note: this notebook needs to be run with a PySpark kernel to work properly!*

In this notebook, we'll use the spark structured streaming for real time feature aggregations and writing them to the online and offline FS:

- Read data from the kafka topic .
- Time window aggregations using spark structured streaming.
- write aggregated feature to the online feature store


## Import necessary libraries

In [1]:
import json

from math import radians

from pyspark.sql.functions import from_json, window, avg,count, stddev, explode, date_format,col, mean, pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType, LongType, IntegerType

from hops import kafka, tls, hdfs

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
1,application_1653266850468_0008,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
# name of the kafka topic to read card transactions from
KAFKA_TOPIC_NAME = "credit_card_transactions"

## Create a stream from the kafka topic


In [3]:
df_read = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka.get_broker_endpoints()) \
  .option("kafka.security.protocol",kafka.get_security_protocol()) \
  .option("kafka.ssl.truststore.location", tls.get_trust_store()) \
  .option("kafka.ssl.truststore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.keystore.location", tls.get_key_store()) \
  .option("kafka.ssl.keystore.password", tls.get_key_store_pwd()) \
  .option("kafka.ssl.key.password", tls.get_trust_store_pwd()) \
  .option("kafka.ssl.endpoint.identification.algorithm", "") \
  .option("startingOffsets", "earliest")\
  .option("subscribe", KAFKA_TOPIC_NAME) \
  .load()

In [4]:
# Define schema to read from kafka topic 
parse_schema = StructType([StructField('tid', StringType(), True),
                           StructField('datetime', StringType(), True),
                           StructField('cc_num', StringType(), True),
                           StructField('category', StringType(), True),
                           StructField('amount', StringType(), True),
                           StructField('latitude', StringType(), True),
                           StructField('longitude', StringType(), True),
                           StructField('city', StringType(), True),
                           StructField('country', StringType(), True)]
                         )

In [5]:
# Deserialise data from and create streaming query
df_deser = df_read.selectExpr("CAST(value AS STRING)")\
                   .select(from_json("value", parse_schema).alias("value"))\
                   .select("value.tid", 
                           "value.datetime", 
                           "value.cc_num", 
                           "value.category", 
                           "value.amount", 
                           "value.latitude", 
                           "value.longitude",
                           "value.city",
                           "value.country")\
                   .selectExpr("CAST(tid as string)", 
                               "CAST(datetime as timestamp)", 
                               "CAST(cc_num as long)", 
                               "CAST(category as string)", 
                               "CAST(amount as double)",
                               "CAST(latitude as double)",
                               "CAST(longitude as double)",
                               "CAST(city as string)",
                               "CAST(country as string)")

In [6]:
df_deser.isStreaming

True

In [7]:
df_deser.printSchema() 

root
 |-- tid: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- category: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)

In [8]:
schema = StructType([StructField('tid', StringType(), True),
                     StructField('datetime', TimestampType(), True),
                     StructField('cc_num', LongType(), True),
                     StructField('category', StringType(), True),
                     StructField('amount', DoubleType(), True),
                     StructField('latitude', DoubleType(), True),
                     StructField('longitude', DoubleType(), True),
                     StructField('city', StringType(), True),
                     StructField('country', StringType(), True),
                     StructField('loc_delta', DoubleType(), True)
                    ]) 

def haversine(pdf):
    import numpy as np
    long = pdf["longitude"].astype(np.float)
    lat = pdf["latitude"].astype(np.float)
    
    long_shifted = long.shift()
    lat_shifted = lat.shift()
    long_shifted = long_shifted.astype(np.float)
    lat_shifted = lat_shifted.astype(np.float)
    
    long_diff = long_shifted - long
    lat_diff = lat_shifted - lat    
    long_diff = long_diff.astype(np.float)
    lat_diff = lat_diff.astype(np.float)

    a = np.sin(lat_diff/2.0)**2
    b = np.cos(lat) * np.cos(lat_shifted) * np.sin(long_diff/2.0)**2
    pdf["loc_delta"]  = 2*np.arcsin(np.sqrt(a + b))

    return pdf

spark.udf.register("radians", radians)

<function radians at 0x7fa9325dd790>

## Create windowing aggregations over 4 hour time window using spark streaming.

In [9]:
# 4 hour window
windowed4hTransactionDF =df_deser \
    .selectExpr("tid", 
                "datetime", 
                "cc_num", 
                "category", 
                "CAST(amount as double)",
                "radians(latitude) as latitude",
                "radians(longitude) as longitude",
                "city",
                "country")\
    .selectExpr("tid", 
                "datetime", 
                "cc_num", 
                "category", 
                "CAST(amount as double)",
                "CAST(latitude as double)",
                "CAST(longitude as double)",
                "city",
                "country")\
    .withWatermark("datetime", "480 minutes") \
    .groupBy(window("datetime", "240 minutes"), "cc_num")\
    .applyInPandas(haversine,schema = schema)\
    .selectExpr(
                "datetime", 
                "cc_num", 
                "category", 
                "CAST(amount as double)",
                "CAST(loc_delta as double)",
                "city",
                "country")\
    .withWatermark("datetime", "10 minutes") \
    .groupBy(window("datetime", "240 minutes"), "cc_num")\
    .agg(mean("amount").alias("trans_volume_mavg"), 
         stddev("amount").alias("trans_volume_mstd"), 
         count("cc_num").alias("trans_freq"),
         mean("loc_delta").alias("loc_delta_mavg")
        )\
    .selectExpr("CAST(cc_num as string)", 
                "trans_volume_mavg", 
                "trans_volume_mstd", 
                "CAST(trans_freq as double)", 
                "loc_delta_mavg", 
                "unix_timestamp(window.end) as datetime")

In [10]:
windowed4hTransactionDF.isStreaming

True

In [11]:
windowed4hTransactionDF.printSchema()

root
 |-- cc_num: string (nullable = true)
 |-- trans_volume_mavg: double (nullable = true)
 |-- trans_volume_mstd: double (nullable = true)
 |-- trans_freq: double (nullable = false)
 |-- loc_delta_mavg: double (nullable = true)
 |-- datetime: long (nullable = true)

### Establish a connection with your Hopsworks feature store.

In [12]:
import hsfs
connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Get feature groups from hopsworks feature store.

In [13]:
transactions_4h_aggs = fs.get_feature_group("transactions_4h_aggs", version = 1)

## Insert streaming dataframes to the online feature group

Now we are ready to write this streaming dataframe as a long living application to the online storage of the other feature group.

In [14]:
query_4h = transactions_4h_aggs.insert_stream(windowed4hTransactionDF)

### Check if spark streaming query is active

In [15]:
query_4h.isActive

True

#### We can also check status of a query and if there are any exceptions thrown.

In [16]:
query_4h.status

{'message': 'Getting offsets from KafkaV2[Subscribe[credit_card_transactions]]', 'isDataAvailable': False, 'isTriggerActive': True}

In [17]:
query_4h.exception()

### Stop queries
If you are running this from a notebook, you can kill the Spark Structured Streaming Query by stopping the Kernel or by calling its `.stop()` method.

In [39]:
query_4h.stop()

## TODO (Davit): gif how to run offline backfill job